<a href="https://colab.research.google.com/github/V1569/Weather-forecasting/blob/main/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -qq cloudflared
!pip install streamlit joblib pandas scikit-learn


In [ ]:
%%writefile weather_data.csv
City,Weather
Bangalore,Sunny
Mumbai,Sunny
Delhi,Sunny
Chennai,Cloudy
Kolkata,Rainy
Hyderabad,Sunny
Pune,Cloudy
Jaipur,Sunny
Lucknow,Sunny
Ahmedabad,Sunny
Bhopal,Cloudy
Surat,Rainy
Kanpur,Sunny
Nagpur,Sunny
Indore,Cloudy
Thiruvananthapuram,Rainy
Patna,Sunny
Vadodara,Cloudy
Coimbatore,Sunny
Agra,Sunny
Mysore,Sunny
Rajkot,Rainy
Varanasi,Sunny
Amritsar,Cloudy
Nashik,Sunny
Meerut,Sunny
Jodhpur,Sunny
Guwahati,Rainy
Visakhapatnam,Cloudy
Srinagar,Snowy


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load dataset
df = pd.read_csv("weather_data.csv")

# Encode City and Weather
le_city = LabelEncoder()
df['City'] = le_city.fit_transform(df['City'])

le_weather = LabelEncoder()
df['Weather'] = le_weather.fit_transform(df['Weather'])

# Train model
X = df[['City']]        # Input: city only
y = df['Weather']       # Output: weather
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X, y)

# Save model and encoders
joblib.dump(model, "weather_model.pkl")
joblib.dump(le_city, "le_city.pkl")
joblib.dump(le_weather, "le_weather.pkl")

print("Model trained successfully!")


In [ ]:
%%writefile app.py
import streamlit as st
import joblib

# Load model and encoders
model = joblib.load("weather_model.pkl")
le_city = joblib.load("le_city.pkl")
le_weather = joblib.load("le_weather.pkl")

st.title("Weather Prediction App 🌤️")

city = st.text_input("Enter City Name")

if st.button("Predict Weather"):
    if city not in le_city.classes_:
        st.error("Invalid!")
    else:
        city_encoded = le_city.transform([city])[0]
        pred = model.predict([[city_encoded]])
        weather = le_weather.inverse_transform(pred)[0]
        st.success(f"The weather in {city} is predicted as: {weather}")


In [ ]:
# Download and install cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


In [ ]:
!cloudflared --version


In [ ]:
# Start Streamlit in background
get_ipython().system_raw("streamlit run app.py --server.port 8501 &")

# Start Cloudflare tunnel
!cloudflared tunnel --url http://localhost:8501
